In [2]:
import glob
import os

# find all .txt files in current folder except already converted ones
txt_files = [f for f in glob.glob("*.txt") if "_hex" not in f and "output" not in f]

for filename in txt_files:
    output_file = filename.replace(".txt", "_hex.txt")
    count = 0
    with open(filename) as fin, open(output_file, "w") as fout:
        for line in fin:
            line = line.strip()
            if not line:
                continue
            try:
                val = int(line)
            except ValueError:
                print(f"⚠️  Skipping invalid line in {filename}: {line}")
                continue
            # wrap to 16-bit two's complement
            if val < 0:
                val = (1 << 16) + val
            fout.write(f"{val:04X}\n")
            count += 1
    print(f"✅ Converted {filename} -> {output_file}  ({count} values)")

✅ Converted comp_R8.txt -> comp_R8_hex.txt  (40 values)
✅ Converted notch_a_q14.txt -> notch_a_q14_hex.txt  (3 values)
✅ Converted comp_R16.txt -> comp_R16_hex.txt  (40 values)
✅ Converted notch_b_q14.txt -> notch_b_q14_hex.txt  (3 values)
✅ Converted comp_R2.txt -> comp_R2_hex.txt  (40 values)
✅ Converted coeffs_fixed_q15.txt -> coeffs_fixed_q15_hex.txt  (228 values)
✅ Converted comp_R4.txt -> comp_R4_hex.txt  (40 values)


In [ ]:
import json
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak

# Load the JSON file
with open("book.json", "r", encoding="utf-8") as f:
    chapters = json.load(f)

# Create a PDF
pdf_filename = "Book.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=A4)
styles = getSampleStyleSheet()
story = []

# Sort chapters by key (c0001, c0002, etc.)
for key in sorted(chapters.keys()):
    chapter_title = key.upper()
    chapter_text = chapters[key]

    # Add title
    story.append(Paragraph(f"<b>{chapter_title}</b>", styles["Heading1"]))
    story.append(Spacer(1, 0.2 * inch))

    # Add text
    story.append(Paragraph(chapter_text.replace("\n", "<br/>"), styles["Normal"]))
    story.append(PageBreak())

# Build PDF
doc.build(story)
print(f"✅ Created: {pdf_filename}")

In [ ]:
import json
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.pdfgen import canvas
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER

# ---------- Custom Page Number Footer ----------
def add_page_number(canvas, doc):
    page_num = canvas.getPageNumber()
    text = f"Page {page_num}"
    canvas.setFont("Helvetica", 9)
    canvas.drawRightString(200 * mm, 10 * mm, text)

# ---------- Load JSON ----------
with open("book.json", "r", encoding="utf-8") as f:
    chapters = json.load(f)

# ---------- Create PDF ----------
pdf_filename = "Book.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=A4,
                        rightMargin=1*inch, leftMargin=1*inch,
                        topMargin=1*inch, bottomMargin=1*inch)

styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY, leading=15))
styles.add(ParagraphStyle(name='Center', alignment=TA_CENTER, fontSize=16, spaceAfter=20))

story = []

# ---------- Optional Title Page ----------
story.append(Spacer(1, 2 * inch))
story.append(Paragraph("<b><font size=24>Release That Witch (Custom Edition)</font></b>", styles['Center']))
story.append(Spacer(1, 0.3 * inch))
story.append(Paragraph("Generated from JSON Chapters", styles['Center']))
story.append(PageBreak())

# ---------- Add Chapters ----------
for key in sorted(chapters.keys()):
    chapter_title = key.upper()
    chapter_text = chapters[key].replace("\n", "<br/>")

    # Chapter Title
    story.append(Paragraph(f"<b>{chapter_title}</b>", styles['Heading1']))
    story.append(Spacer(1, 0.2 * inch))

    # Chapter Body (multi-page text automatically handled)
    story.append(Paragraph(chapter_text, styles['Justify']))
    story.append(PageBreak())

# ---------- Build PDF ----------
doc.build(story, onLaterPages=add_page_number, onFirstPage=add_page_number)
print(f"✅ Created: {pdf_filename}")